## Test trained model

In [1]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Embedding,Bidirectional,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tree_stem import stem_word, word_to_vec
from tokenize_uk import tokenize_words
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [2]:
tf.__version__

'2.14.0'

In [3]:
df = pd.read_csv('messages_dataset.csv') # load dataset
class_name = ['economics', 'politics', 'sports', 'entertainment', 'technology']
ukrainian_stopwords = list(pd.read_csv("stopwords_ua.txt", header=None, names=['stopwords']).stopwords)

In [ ]:
def get_dataset_equally(df):
    grouped = df.groupby(['topic'])
    smallest = grouped.count().min().values
    try: # Pandas 1.1.0+
        return grouped.sample(smallest)
    except AttributeError: # Pre-Pandas 1.1.0
        return grouped.apply(lambda df: df.sample(smallest))
df = get_dataset_equally(df)
df.topic.value_counts()

In [11]:

# clean dataset messages
def clean_message(message):
    text = re.sub("https?:\/\/[\w+.\/]+", " ", str(message))
    text = re.sub('[^a-zабвгґдеєжзиіїйклмнопрстуфхцчшщьюя]', ' ', str(text)).lower()
    sen = re.sub(' +',' ',str(text))
    words = tokenize_words(sen)
    sen = []
    for word in words:
        if word not in ukrainian_stopwords:
            try:
                sen.append(stem_word(word))
            except:
                sen.append(word)

    text = ' '.join(sen)
    return text.strip()


In [21]:
corpus = [] 
x = df.message
y = df.topic

for msg in x:
    corpus.append(clean_message(msg))
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)

In [22]:
#load model
model = tf.keras.models.load_model("lstm-bidir-250words-16batch-07-0.95.keras")


In [23]:
text_input = ["""Нове кафе Cafe mariia ✨
📍Де?
Павлівська, 26/41 (10 хв від Цирку)
Тут шумно, людно і дуже живо. Таким і має бути міське кафе.
Відчуття, ніби всі найпрекрасніші люди Києва зібрались в одному місці. Творчі, легкі, талановиті, з летючим щасливим поглядом та мріями про велике в розмовах. Напрочуд прекрасно Марії вдалось зібрати власне ком’юніті, про яке, мабуть, мріяв би кожний заклад✨
Приміщення та меблі перейшли у спадок від кафе Зоря, яка тут прожила менш як рік. З нового: з’явилась велика пекарська вітрина та трохи змінилось розташування посадки.
Меню Марії — комфортік. Тут легко зорієнтуватись, бо всі позиції давно знайомі. Але класно, що у створенні страв допомагали шефи і тому навіть звичні рецепти заграли новими фарбами. Наприклад, на сніданок подають телячий язик, а ввечері готують крудо з фенхелем.
"""]
opt_len = 250

# clean input message
text = clean_message(text_input)

print(text)

sequence = tokenizer.texts_to_sequences([text])
print(sequence)
embedding_text = pad_sequences(sequence,padding='post',maxlen=opt_len)
print(text_input)
print('topic is: {}'.format(class_name[np.argmax(model.predict([embedding_text]))]))

ов каф afe mariia n n авлівськ х ир n ут шумно людно жив ак ма міськ каф n ідчутт найпрекрасніш иєв з міс ворч легк талановит летюч щаслив погляд мрі велик розмов апрочуд арі вда зібра к юніт мрі заклад n риміщенн мебл перей спад каф  прож менш нов яв велик пекарськ вітрин змін розташуванн посад n ен арі комфортік ут лег зорієнту позиці знайом класно створенн стра допомага шеф звичн рецепт загра нов фарб априклад снідан пода теляч язик ввечер готу круд фен n
[[6688, 1705, 1, 56509, 9278, 9278, 1, 448, 1, 9278, 7992, 1, 1, 804, 12721, 20, 803, 1705, 9278, 1, 37019, 1, 14, 49, 1, 1163, 2765, 10516, 3337, 1109, 974, 46, 1128, 1, 19619, 530, 550, 666, 36472, 974, 533, 9278, 1, 6462, 3618, 3547, 1705, 8208, 561, 19, 10232, 46, 1, 14259, 70, 3541, 573, 9278, 11633, 19619, 1, 7992, 1590, 29165, 363, 1278, 10137, 377, 3994, 731, 4244, 3287, 3917, 8894, 19, 7671, 1, 5128, 607, 23018, 11227, 3606, 464, 1, 33048, 9278]]
['Нове кафе Cafe mariia ✨\n📍Де?\nПавлівська, 26/41 (10 хв від Цирку)\nТут шум